# Imports

In [2]:
#Import FISHscale
import sys
sys.path.insert(0, "/Users/admin/OneDrive - KI.SE/Projects")#FISHscale/")
from FISHscale.utils import dataset
from FISHscale.graphNN.graph_utils import GraphData

#Import other libraries
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import LineCollection
import numpy as np
import pickle
import os

#Get data path
cur_path = os.path.dirname(os.getcwd())
data_path = os.path.join(cur_path, 'Example_data')

# Load data
Data can be loaded from .csv or .parquet files, where .parquet files are the most efficient. These files should at least have a column for the X and Y coordinates of the points and a colum with gene names. The names of these columns can be passed to the loading function with the parameters: `x_label`, `y_label` & `gene_label`. Other colums can be loaded using the `other_columns` parameter.  

  
The unit is also handled explicitly by giving the unit scale using the `pixel_size` parameter. For our data the XY coordinates are in pixel units which are 0.18 micrometer. Change it according to the settings of your system. If your data is already in a specific unit you can just pass this, like `1 micrometer` or `1 meter`.

The color of the genes can be predefined by giving a dictionary with colors as input. If not defined, FISHscale will generate a new color dictionary and save it fot later use.
  
The first time you load a dataset the data will be parsed which can take some time. However, the next time you load it, it will be fast.

In [5]:
file_name = os.path.join(data_path, 'Mouse_448_example_data.parquet')
color_dictionary = pickle.load(open(os.path.join(data_path, 'Mouse_448_color_dict.pkl'), 'rb'))
tissue_polygon = np.genfromtxt(os.path.join(data_path, 'Outside_polygon_Mouse_448.csv'), delimiter=',')[1:, 1:]

d0 = dataset.Dataset(file_name,
                     x_label = 'c_px_global_stitched',
                     y_label = 'r_px_global_stitched',                
                     gene_label = 'decoded_genes',
                     pixel_size = '0.18 micrometer',
                     color_input = color_dictionary,
                     exclude_genes = ['Control1', 'Control2', 'Control3', 'Control4', 'Control5', 'Control6', 'Control7', 'Control8', 'Cre'],
                     polygon = tissue_polygon,
                     verbose = True,
                     reparse=True)

adding to metadata


100%|█████████████████████████████████████████| 440/440 [00:07<00:00, 61.68it/s]


    Loaded: Mouse_448_example_data


You can inspect your data by quickly plotting a few genes.

# GraphSAGE

GraphData class will transform the Dataset into a graph that will be used to train the GraphData.model (GraphSAGE)

In [ ]:
GD = GraphData(d0,
               analysis_name='Unsupervised_GraphSAGE',
               train_p=0.1,
               num_workers=6,
               aggregator='pool',
              )

model is in:  cpu
Chosen dist: 12.594856092869993


/Users/admin/anaconda3/envs/py38/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory Unsupervised_GraphSAGE_2021-12-15-153116 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
100%|█████████████████████████████████████████| 440/440 [00:59<00:00,  7.38it/s]


Building graph...


100%|████████████████████████████| 8871209/8871209 [00:06<00:00, 1363493.10it/s]


Find neighbors below distance: 12.594856092869993


100%|███████████████████████████████| 8871209/8871209 [47:10<00:00, 3133.68it/s]


Train GraphSAGE network for 10 epochs. If GPU is available, GraphData will use it.

In [ ]:
GD.train(max_epochs=10)

Generate molecule embeddings that will be saved in a folder with analysis_name defined in GraphData. Visualize the embedding using umap.

In [ ]:
GD.get_latents()
GD.get_umap()

# Adding Dask Attributes for visualization

Clustering of generated molecule embeddings

In [ ]:
from sklearn.cluster import MiniBatchKMeans
embedding = np.load('Unsupservised_GraphSAGE/latent_unlabelled.npy')
km = MiniBatchKMeans(n_clusters=150).fit_predict(embedding)

To generate a fast visualization of large points we sacrifica disk space, data is written again organized by the new column introduced to the dataset.

In [ ]:
d0.add_dask_attribute('KMeans',km.astype('str'),include_genes=True)


Then we can plot by new attribute very fast for millions of points

In [ ]:
d0.visualize(['KMeans'])